In [1]:
from cl_v2 import trader
from cl_v2 import back_klines
from cl_v2 import cl
from cl_v2 import kcharts
from cl_v2 import rd
from cl_v2.my import strategy_demo
import time
import datetime
import threading
import pickle


In [2]:
# 量化配置
start_datetime = '2021-10-10 00:00:00'
end_datetime = '2021-10-20 00:00:00'
market = 'hk'
codes = ['HK.09961', 'HK.00981', 'HK.00189', 'HK.09922', 'HK.06098', 'HK.06186', 'HK.00175']
frequencys = ['30m']

STR_Class = strategy_demo.Strategy_Demo()

In [3]:
traders = {}
bks = {}
_start_datetime = datetime.datetime.strptime(start_datetime, '%Y-%m-%d %H:%M:%S')
_end_datetime = datetime.datetime.strptime(end_datetime, '%Y-%m-%d %H:%M:%S')


for code in codes:
    bks[code] = back_klines.BackKlines(market, code, _start_datetime, _end_datetime, frequencys)
    bks[code].start()
    traders[code] = trader.Trader(code, is_stock=True, is_futures=False, log=print)
    traders[code].pos_space = 3
    traders[code].set_strategy(STR_Class)
    
    
def run(bk, td):
    start = time.time()
    while True:
        ok = bk.next(frequencys[-1])
        if ok is False:
            break

        td.run(bk.code, bk.cl_datas)
    td.end()
    end = time.time()
    print('%s Total Run: %.4f s' % (td.name, end - start))

# 多线程运行程序
threads = {}
for code in codes:
    threads[code] = threading.Thread(target=run, args=(bks[code], traders[code]))
    threads[code].setDaemon(True)
    threads[code].start()


print('Done')

Code HK.09961 F 30m len 814
Code HK.00981 F 30m len 814
Code HK.00189 F 30m len 814
Code HK.09922 F 30m len 814
Code HK.06098 F 30m len 814
Code HK.06186 F 30m len 814
Code HK.00175 F 30m len 814
[HK.09961 - 2021-10-11 10:30:00] // 3buy 做多买入（248.0 - 399.1935），原因： 高级别出现 3buy (TD: True Done: True)
[HK.00981 - 2021-10-11 10:30:00] // 2buy 做多买入（22.1 - 4479.638），原因： 高级别出现 2buy (TD: True Done: True)Done

[HK.00189 - 2021-10-11 13:30:00] // 1buy 做多买入（14.88 - 6653.2258），原因： 高级别出现 1buy (TD: True Done: True)
[HK.00189 - 2021-10-12 10:00:00] // 1buy 平仓做多（14.3 - 6653.2258） 盈亏：-3858.8709640000015  (-3.90%)，原因： 止损
[HK.09922 - 2021-10-11 15:00:00] // 1buy 做多买入（21.55 - 4593.9675），原因： 高级别出现 1buy (TD: True Done: True)
[HK.09961 - 2021-10-12 10:00:00] // 3buy 平仓做多（249.8 - 399.1935） 盈亏：718.5482999999949  (0.73%)，原因： 高级别笔（TD: True Done: True），多仓清仓
[HK.00981 - 2021-10-12 14:00:00] // 2buy 平仓做多（21.65 - 4479.638） 盈亏：-2015.8371000000188  (-2.04%)，原因： 止损
[HK.06098 - 2021-10-15 10:30:00] // 3buy 做多买入（62.8 - 1576

In [4]:
# 打印现在执行到的时间情况
for code in codes:
    print('%s run datetime : %s' % (code, bks[code].now_date))

HK.09961 run datetime : None
HK.00981 run datetime : None
HK.00189 run datetime : None
HK.09922 run datetime : None
HK.06098 run datetime : None
HK.06186 run datetime : None
HK.00175 run datetime : None


In [5]:
trader.traders_result(traders.values())

+------------+------+------+--------+---------+---------+----------+----------+----------+--------+
|   买卖点   | 成功 | 失败 |  胜率  |   盈利  |   亏损  |  净利润  | 平均盈利 | 平均亏损 | 盈亏比 |
+------------+------+------+--------+---------+---------+----------+----------+----------+--------+
|  一类买点  |  1   |  1   | 50.0%  | 5283.06 | 3858.87 | 1424.19  | 5283.06  | 3858.87  | 1.3691 |
|  二类买点  |  0   |  3   |  0.0%  |    0    | 4639.36 | -4639.36 |    0     | 1546.45  |   0    |
| 类二类买点 |  1   |  0   | 100.0% | 3186.21 |    0    | 3186.21  | 3186.21  |    0     |   0    |
|  三类买点  |  1   |  1   | 50.0%  |  718.55 | 2049.36 | -1330.81 |  718.55  | 2049.36  | 0.3506 |
| 类三类买点 |  0   |  0   |   0%   |    0    |    0    |    0     |    0     |    0     |   0    |
|  一类卖点  |  0   |  0   |   0%   |    0    |    0    |    0     |    0     |    0     |   0    |
|  二类卖点  |  0   |  0   |   0%   |    0    |    0    |    0     |    0     |    0     |   0    |
| 类二类卖点 |  0   |  0   |   0%   |    0    |    0    |    0

In [6]:
# 将回测对象保存到 Redis 中，以便于网页进行查看
strategy_back_save_key = 'hk_stock_stratege_demo_' + datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
print('Strategy key : ', strategy_back_save_key)
p_obj = pickle.dumps(traders)
rd.strategy_save(strategy_back_save_key, p_obj)

Strategy key :  hk_stock_stratege_demo_2021_12_18_16_38_39


1

In [6]:
# 需要开启保存k线功能
renders = {}
for code in traders.keys():
    td = traders[code]
    renders[td.name] = {}
    for f in frequencys:
        renders[td.name][f] = td.show_kline(td.name, show_frequency=f)

In [7]:
renders[codes[0]]['30m']

In [9]:
print('交易对,买卖点,开仓时间,平仓时间,类型,价格,数量,止损,盈亏,最大盈利,最大亏损,开仓信息,平仓信息,高级别买卖点,高级别停顿,高级别同类中枢,低级别分解买卖点,低级别扩展买卖点,低级别停顿,低级别同类中枢')
for td in traders.values():
    if td.name not in td.positions_history:
        continue
    for p in td.positions_history[td.name]:
        print('%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s' % (
            td.name, p.mmd, p.open_datetime, p.close_datetime, p.type, p.price, p.amount, p.loss_price, 
            p.profit_rate, p.max_profit_rate, p.max_loss_rate, p.open_msg.replace(',', ''), p.close_msg.replace(',', '')))

交易对,买卖点,开仓时间,平仓时间,类型,价格,数量,止损,盈亏,最大盈利,最大亏损,开仓信息,平仓信息,高级别买卖点,高级别停顿,高级别同类中枢,低级别分解买卖点,低级别扩展买卖点,低级别停顿,低级别同类中枢
HK.09961,3buy,2021-10-11 10:30:00,2021-10-12 10:00:00,做多,248.0,1317.3387,241.8,0.73,1.2097,0,高级别出现 3buy (TD: True Done: True),高级别笔（TD: True Done: True），多仓清仓
HK.00981,2buy,2021-10-11 10:30:00,2021-10-12 14:00:00,做多,22.1,14782.8054,21.7,-2.04,0.6787,-1.81,高级别出现 2buy (TD: True Done: True),止损
HK.00981,l2buy,2021-10-18 15:00:00,2021-10-19 16:00:00,做多,21.75,14920.7679,21.45,3.22,3.908,0,高级别出现 l2buy (TD: True Done: True),退出
HK.00189,1buy,2021-10-11 13:30:00,2021-10-12 10:00:00,做多,14.88,21955.6452,14.5,-3.9,1.0753,-2.957,高级别出现 1buy (TD: True Done: True),止损
HK.09922,1buy,2021-10-11 15:00:00,2021-10-12 14:00:00,做多,21.55,15160.0928,21.7655,5.34,8.3527,0,高级别出现 1buy (TD: True Done: True),高级别笔（TD: True Done: True），多仓清仓
HK.09922,2buy,2021-10-12 16:00:00,2021-10-15 13:30:00,做多,22.6,14707.7755,22.25,-2.21,3.9823,-1.1062,高级别出现 2buy (TD: True Done: True),止损
HK.09922,2buy,2021-10-18 10:00:00,2021-10-1

In [10]:
kcharts.render_cash(traders[codes[0]].balances_history[codes[0]])

In [11]:
bk.show_klines

NameError: name 'bk' is not defined